
# 🔹 УСТАНОВКА + ВЫБОР МОДЕЛИ

---

##  Установка (ТЕРМИНАЛ)

```bash
sudo apt update
sudo apt upgrade -y
sudo apt install -y curl git python3 python3-pip python3-venv
```


##  Создание папки проекта

```bash
mkdir ml_championship
cd ml_championship
```



##  Виртуальное окружение Python

```bash
python3 -m venv venv
source venv/bin/activate
```


## Установка Jupyter Notebook

```bash
pip install notebook jupyterlab
```



## Установка Ollama

```bash
curl -fsSL https://ollama.com/install.sh | sh
```

Проверка:

```bash
ollama --version
```



##  Запуск Ollama

```bash
ollama serve
```

❗ Терминал не закрывать


## Скачивание модели

```bash
ollama pull mistral:7b
```



##  Выбор модели 

| Модель         | Размер  | RAM     | Для чего лучше всего |
| -------------- | ------- | ------- | -------------------- |
| **mistral:7b** | ~4.1 GB | 8–10 GB | универсал, RAG       |
| llama2:7b      | ~3.8 GB | 8 GB    | простой чат          |
| llama2:13b     | ~7.3 GB | 16 GB   | сложные ответы       |
| gemma:2b       | ~1.5 GB | 4 GB    | слабые ПК            |
| gemma:7b       | ~4 GB   | 8 GB    | тексты, анализ       |
| codellama:7b   | ~4 GB   | 8 GB    | код                  |

---

# 🔹 ДООБУЧЕНИЕ (FINE-TUNING)

---

##  Как Ollama делает fine-tuning

Ollama принимает:

```
ollama train <базовая_модель> <файл_с_данными> <новое_имя_модели>
```

Что происходит внутри:

1. данные читаются построчно
2. модель дообучается на них
3. создаётся **новая модель**, старая не портится


##  Подготовка данных



##  Запуск fine-tuning (ТЕРМИНАЛ)


```bash
ollama train mistral:7b data/text.txt mistral_text_ft
```

```bash
ollama train mistral:7b data/table.csv mistral_table_ft
```


##  Проверка дообученной модели (В JUPYTER)

```python
from ollama import chat

response = chat(
    model="mistral_text_ft",
    messages=[{"role": "user", "content": "Что такое RAG?"}]
)

print(response["message"]["content"])
```


---

# 🔹 API (FASTAPI)

---

Файл:

```
ml_championship/api.py
```


## Код API

```python
from fastapi import FastAPI
from pydantic import BaseModel
from ollama import chat

app = FastAPI()

class Query(BaseModel):
    text: str

@app.post("/ask")
def ask(q: Query):
    response = chat(
        model="mistral:7b",
        messages=[{"role": "user", "content": q.text}]
    )
    return {"answer": response["message"]["content"]}
```


## Запуск API (В ТЕРМИНАЛЕ)

```bash
cd ml_championship
source venv/bin/activate
uvicorn api:app --reload
```

Открыть в браузере:

```
http://127.0.0.1:8000/docs
```

---

# 🔹 TELEGRAM-БОТ

---

##  Создание бота

1. Telegram → `@BotFather`
2. `/start`
3. `/newbot`
4. сохранить TOKEN


## Код Telegram-бота

```python
from telegram.ext import Application, MessageHandler, filters
from ollama import chat

TOKEN = "PASTE_YOUR_TOKEN_HERE"

async def handle_message(update, context):
    user_text = update.message.text

    response = chat(
        model="mistral:7b",
        messages=[{"role": "user", "content": user_text}]
    )

    await update.message.reply_text(response["message"]["content"])

app = Application.builder().token(TOKEN).build()
app.add_handler(MessageHandler(filters.TEXT, handle_message))
app.run_polling()
```


## Запуск бота (В ТЕРМИНАЛЕ)

```bash
cd ml_championship/bot
source ../venv/bin/activate
python bot.py
```
---

# 🔹 ДООБУЧЕНИЕ RAG вариант две модели (Embedding + LLM) особенно полезно если дан огромный текст не формата вопрос - ответ

--- 

1. Модель для эмбеддингов
 - Специализированная embedding‑модель (например nomic-embed-text)
Задача: превратить куски текста в векторы для поиска по смыслу
 - Не генерирует текст, только числовые представления

2. Модель для генерации ответа
 - Классическая LLM (например mistral:7b)
Задача: на основе найденного контекста красиво и связно ответить на вопрос

🔹 Пошаговая схема
```bash
Пользователь задаёт вопрос
        │
        ▼
Embedding-модель → переводит все чанки текста в векторы
        │
        ▼
Сравнение векторов → выбираем топ-k релевантных чанков
        │
        ▼
LLM (например Mistral) → формируем красивый ответ с использованием этих чанков
        │
        ▼
Пользователь получает ответ
```
## Установка

``` bash
ollama pull nomic-embed-text
# если более точную:
# ollama pull mxbai-embed-large
```


``` bash
ollama pull mistral:7b
```

## Пример кода (название файла text.txt)

```python
from ollama import embeddings, chat
import numpy as np

# -----------------------------
# 1) читаем текст и разбиваем на чанки
# -----------------------------
with open("../data/document.txt", "r", encoding="utf-8") as f:
    text = f.read()

def split_text(text, chunk_size=400):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

chunks = split_text(text)

# -----------------------------
# 2) создаём эмбеддинги для всех чанков
# -----------------------------
chunk_embeddings = []
for chunk in chunks:
    emb = embeddings(
        model="nomic-embed-text",  # embedding-модель
        prompt=chunk
    )
    chunk_embeddings.append(emb["embedding"])

# -----------------------------
# 3) функция косинусного сходства
# -----------------------------
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def embeddings_fn(text):
    emb = embeddings(model="nomic-embed-text", prompt=text)
    return emb["embedding"]

# -----------------------------
# 4) поиск топ-k чанков
# -----------------------------
def retrieve_chunks(question, chunks, embeddings, top_k=3):
    q_emb = embeddings_fn(question)
    similarities = [cosine_similarity(q_emb, e) for e in embeddings]
    top_indices = np.argsort(similarities)[-top_k:]
    return [chunks[i] for i in top_indices]

# -----------------------------
# 5) собираем контекст и формируем prompt для LLM
# -----------------------------
def build_prompt(context_chunks, question):
    context = "\n\n".join(context_chunks)
    prompt = f"""
Ты — ассистент, который отвечает на вопрос,
используя ТОЛЬКО информацию из контекста.

Контекст:
{context}

Вопрос:
{question}

Ответ:
"""
    return prompt

# -----------------------------
# 6) итоговая функция RAG с двумя моделями
# -----------------------------
def rag_answer(question):
    relevant_chunks = retrieve_chunks(question, chunks, chunk_embeddings)
    prompt = build_prompt(relevant_chunks, question)
    response = chat(
        model="mistral:7b",  # LLM для красивого ответа
        messages=[{"role": "user", "content": prompt}]
    )
    return response["message"]["content"]

# -----------------------------
# 7) тест
# -----------------------------
print(rag_answer("Какие основные характеристики больших данных?"))
```
